In [1]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox

In [ ]:
window=tk.Tk()
window.title("handwritten digit recognition")
l1=tk.Label(window,text="Digit",font=('Algerian',20))
l1.place(x=5,y=0)
t1=tk.Entry(window,width=20, border=5)
t1.place(x=150,y=0)


def screen_capture():
    import pyscreenshot as ImageGrab
    import time
    import os
    
    s1=t1.get()
    os.chdir("C:/Users/mohit/OneDrive/Desktop/ML/Untitled Folder/captured_images")
    os.mkdir(s1)
    os.chdir("C:/Users/mohit/OneDrive/Desktop/ML/Untitled Folder/")
    
    image_folder = "captured_images/"+s1+"/"
    time.sleep(15)
    for i in range (0,5):
        time.sleep(8)
        im = ImageGrab.grab(bbox=(372,512,884,884))
        print("saved......",i)
        im.save(image_folder+str(i)+'.png')
        print("clear screen now and redraw now.....")
    messagebox.showinfo("Result","Capturing screen is Completed !!")
    
b1=tk.Button(window,text="1. Open Paint and capture the screeen", font=('Algerian',15),bg="orange",fg="black",command=screen_capture) 
b1.place(x=5,y=50)

def generate_dataset():
    import cv2
    import csv
    import glob

    header = ["label"]
    for i in range(0,784):
        header.append("pixel" + str(i))
    with open('dataset.csv','a') as f:
        writer=csv.writer(f)
        writer.writerow(header)
    
    for label in range(10):
        dirList=glob.glob("captured_images/"+ str(label)+"/*.png")
    
        for img_path in dirList:
            im=cv2.imread(img_path)
            im_gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
            im_gray=cv2.GaussianBlur(im_gray,(15,15),0)
            roi=cv2.resize(im_gray,(28,28),interpolation=cv2.INTER_AREA)
        
            data=[]
            data.append(label)
            rows,cols=roi.shape
        
            for i in range(rows):
                for j in range(cols):
                    k=roi[i,j]
                if k>100:
                    k=1
                else:
                    k=0
                data.append(k)
            
            with open('dataset.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(data)
    messagebox.showinfo("Result","Ganerating Dataset is Completed !!")
    
b2=tk.Button(window,text="2. Generate dataset and load it ", font=('Algerian',15),bg="pink",fg="blue",command=generate_dataset)
b2.place(x=5,y=100)

def train_save_accuracy():
    import pandas as pd
    from sklearn.utils import shuffle
    from sklearn.impute import SimpleImputer
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC
    from sklearn import metrics
    import joblib

    data = pd.read_csv('dataset.csv')
    data = shuffle(data)
    X = data.drop(["label"], axis=1)
    Y = data["label"]

       # Preprocess the data to handle missing values
    imputer = SimpleImputer(strategy='mean')  # Replace missing values with the mean
    X = imputer.fit_transform(X)

    train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2)

    classifier = SVC(kernel="linear", random_state=6)
    classifier.fit(train_x, train_y)
    joblib.dump(classifier, "model/digit_recognizer")

    prediction = classifier.predict(test_x)
    acc = metrics.accuracy_score(prediction, test_y)
    messagebox.showinfo("Result", f"Your accuracy is {acc}")

    
b3=tk.Button(window,text="3. train the model, save it and calculate accuracy", font=('Algerian',15),bg="green",fg="white",command=train_save_accuracy)
b3.place(x=5,y=150)

def prediction():
    import joblib
    import cv2
    import numpy as np
    import time
    import pyscreenshot as ImageGrab

    model=joblib.load("model/digit_recognizer")
    images_folder="img/"
    time.sleep(10)
    while True:
        img=ImageGrab.grab(bbox=(372,512,884,884))

        img.save(images_folder+"img.png")
        im=cv2.imread(images_folder+"img.png")
        im_gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im_gray=cv2.GaussianBlur(im_gray,(15,15),0)


        ret,im_th =cv2.threshold(im_gray,100,255,cv2.THRESH_BINARY)
        roi=cv2.resize(im_th, (28,28), interpolation =cv2.INTER_AREA)

        rows,cols=roi.shape
        
        X = []


        for i in range(rows):
                for j in range(cols):
                    k = roi[i,j]
                    if k>100:
                        k=1
                    else:
                        k=0
                    X.append(k)

        predictions = model.predict([X])
        print("Prediction:",predictions[0])
        cv2.putText(im, "Prediction is " +str(predictions[0]), (20,20), 0, 0.8,(0,255,0),2,cv2.LINE_AA)

        cv2.startWindowThread()
        cv2.namedWindow("Result")
        cv2.imshow("Result",im)
        cv2.waitKey(100000)
        if cv2.waitKey(1)==13:
            break
    cv2.destroyAllWindows()
    
    
    
b4=tk.Button(window,text="4. Live Prediction", font=('Algerian',15),bg="white",fg="red",command=prediction)
b4.place(x=5,y=200)

window.geometry("600x300")
window.mainloop()

Prediction: 6
Prediction: 6
Prediction: 1
Prediction: 0
Prediction: 0
Prediction: 0
Prediction: 0
Prediction: 0
Prediction: 0
